# FIT5196 Task 3 in Assessment 1
#### Student Name: Yicheng Zhang
#### Student ID: 27699641

Date: 02/04/2018

Version: 1.0

Environment: Python 3.6.3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python 3.6) 
* re (for regular expression, included in Anaconda Python 3.6) 
* html(for html escape characters parse,included in Anaconda Python 3.6)
* json(for JSON data convert,included in Anaconda Python 3.6)
* pprint(for better display of List,included in Anaconda Python 3.6)

## 1.  Import libraries

In [ ]:
import re
import html
import json
from pprint import pprint

## 2. Parse XML File

[XML](https://en.wikipedia.org/wiki/XML) as the defination in wiki:
> XML is a markup language that defines a set of rules for encoding documents in a format that is both human-readable and machine-readable

There are many packages in python can be used to prase the XML data, however, This task require us to know how to use the regular experssion without using only third party xml packages to find the structure in the XML file, match the data in XML file, finally convert these data to [json](https://en.wikipedia.org/wiki/JSON)
- - -
Regular expression:
>Regular expression is, in theoretical computer science and formal language theory, a sequence of characters that define a search pattern. Usually this pattern is then used by string searching algorithms for "find" or "find and replace" operations on strings

- - -
After checking the defination of XML and regular expression, we can have the first idea about this task. First, we should extract the xml file as String, then match the pattern with groups, finally store the groups as JSON format.

- - -
First, observe the XML file's structure:
```XML
<Terms>
    <Term>
        <title>XXXX</title>
        <Description>XXXX</Description>
        <RelatedTerms>
            <Term>
               <title>XXXX</title>
               <Relationship>XXXX</Relationship>
            </Term>
             ...
        </RelatedTerms>
    </Term>
    ...
</Terms>
```
There are many elements in a tree root named `Terms`,each elements (`Term` in XML file) have serveral attributes:

`title` `Description` `RelatedTerms`

The RelatedTerms also have one or more elements `Term` in it. There are some data only have `title` and `description` without `RelatedTerms`
- - -
Then we check about the expected output:`JSON` file and its structure[3](https://docs.python.org/2/library/json.html):

|**Python**|**JSON**| 
|------|------|
|dict|object|
|list, tuple|array|
|str, unicode|string|

After know the desired output structure, we can create our own data structure by using python, and convert it to JSON using `json` package.
- - - 
Finally, we can use the Python using different data structure to store the data from the XML file and save the data to the JSON file.The macroscopic view of the dataset should be:
```python
{"Terms": [{Term},{Term},...]}
```
the data should store in a `dictionary`,with values which are elements store in the `list`. Each Element should store in `dictionary`,use the attributes as the key, and value in that attributes as the value:
```python
{"Title": 'XXXX',"Description": 'XXXX',"RelatedTerms": [{Term},{Term},...]}
```
similarly, the `RelatedTerms`, should be a `list` to store `dictionaries` of each `relatedterms`:
```python
{"Title": 'XXXX',"Relationship": 'XXXX'}
```

**After analyzing the files and data structure, we can now begin to parse the data**

### 2.1. Load XML File

**Load the XML file as String object which can be matched by regular expression**

In [ ]:
with open('australian-sport-thesaurus-student.xml',encoding='utf-8') as xml_object: #load file as object
    lines = xml_object.readlines() #read every lines in file
xml_string = '' # initial string object
for line in lines: 
    xml_string += line.lstrip()  #remove the left indent in xml file
xml_string #check the string obejct

### 2.2. Check XML Data, & Cleaning

Because the XML file using the markdown language, when it was converted to String, There are some special escape character[4](https://www.w3schools.com/html/html_entities.asp)

|**XML**|**String**| 
|------|------|
|&amp|'&'|
|&#13|Retrun '\r'|
|&lt|'<'|
|&gt|'>'|

This type of escape character should be convert to the normal string, to finish this job, we choose a packages named `html` to convert the escape character to normal string.

**Check if there are escape character in our string**

In [ ]:
print('&amp' in xml_string)
print('&#13' in xml_string)
print('&lt' in xml_string)
print('&gt' in xml_string) #check escape charater in string

**Remove the unnecessary Line Feed character**

In [ ]:
xml_string = xml_string.replace('\n','') 
xml_string[0:500] 

**Using the method named `unescape()` in `html` package to convert escape character to normal string**

In [ ]:
xml_string = html.unescape(xml_string)
print('&amp' in xml_string)
print('#13' in xml_string)
print('&lt' in xml_string)
print('&gt' in xml_string)

**Check the string after processing**

In [ ]:
xml_string

It can be found there are still some character need to be handle,One critical sample is:

`\r` in a String is used for `Carriage Return`

When we store the data, this separator is unnecessary, We can replace these character with one space for string divid purpose.(It also can be replace with empty)

**Check the `Carriage Return` character in the String**

In [ ]:
print('\r' in xml_string)

`\t` in a String is used for `Move to next Tab`
**Check the `Table` character in the String**

In [ ]:
print('\t' in xml_string)

**replace `Carriage Return` character with one space**

In [ ]:
xml_string = xml_string.replace('\r',' ')
print('\r' in xml_string)

**replace `Table` character with one space**

In [ ]:
xml_string = xml_string.replace('\t',' ')
print('\t' in xml_string)

**Check the string after processing**

In [ ]:
xml_string

### 2.3. Process the data

After cleaning the data, we could try to use the regular expression to match the data, divide to different groups.
___
Refer to the XML file's structure:
```XML
<Terms>
    <Term>
        <title>XXXX</title>
        <Description>XXXX</Description>
        <RelatedTerms>
            <Term>
               <title>XXXX</title>
               <Relationship>XXXX</Relationship>
            </Term>
             ...
        </RelatedTerms>
    </Term>
    ...
</Terms>
```
First, we need all the content between:
```XML
<Term>
    content
</Term>
    ...
```
Then, if there are relatedterms in this data, we need split them and store them into a list:
```XML

<RelatedTerms>
    content1
    content2
    ...
</RelatedTerms>
```
___
Using the regular expression `findall(r,String)` method, can divide every matches in `tuple` store in a `list`

One of benfit for the tuples in list is that this data can be easily convert to dictionaries using `dict(list[tuple,tuple...])`

**Try to divide data into groups **

In [ ]:
groups = re.findall('<Term>(.*?)(?:<RelatedTerms>(.*?)</RelatedTerms>)?</Term>', xml_string) #match Content in <Term> and <RelatedTerms>
groups[0:5]

** Check the content for each group Using the 5th data as a sample**

In [ ]:
print(groups[4],'\n')
print(groups[4][0],'\n')
print(groups[4][1],'\n')

Each data(`Term`) store in one tuple, `tuple[0]` is the `Term`'s `Title`&`Description`, `tuple[1]` is the `RelatedTerms`

**Try to match the relatedterms in the above sample**

In [ ]:
relatedmatch = re.findall('<Term><(.*?)>(.*?)</.*?><(.*?)>(.*?)</.*?>?</Term>',groups[4][1])
pprint(relatedmatch)
print('\n')
print(relatedmatch[0][0:2],'\n')
print(relatedmatch[0][2:4],'\n')

Now we get the groups in `Tuple` structure store in a `List` as desired, For the `RelatedTerms`, We should divide the RelatedTerms and store them to `dictionaries`, finally store `dictionaries` to `List`
___
From the above results, If we want to store each RelatedTerm to a dictionary,For the data like:
```python
('Title', '1 Kilometre TT', 'Relationship', 'Used For')
```
we should first store them like:
```python
[('Title', '1 Kilometre TT'),( 'Relationship', 'Used For')]
```
Then convert it to dictionary.

**Try to convert the data**

In [ ]:
root = [] #create list store dictionaries
elements = re.findall('<Term>(.*?)(?:<RelatedTerms>(.*?)</RelatedTerms>)?</Term>', xml_string) #match Content in <Term> and <RelatedTerms>
for element in elements:
    term = re.findall('<(.*?)>(.*?)</.*?>',element[0]) #match Content in <Term></Term>
    termdic = dict(term)
    if element[1] is not '':   
        relatedterms = re.findall('<Term><(.*?)>(.*?)</.*?><(.*?)>(.*?)</.*?>?</Term>',element[1])#match Terms in <RelatedTerms>
        relatedresult = []
        for related in relatedterms:
            tuple1 = related[0:2] 
            tuple2 = related[2:4] #split each Terms in relatedTerms to tuples
            templist = [tuple1,tuple2]
            tempdict = dict(templist) #create dictionaries store relatedTerms
            relatedresult.append(tempdict)
        termdic['RelatedTerms'] = relatedresult
    root.append(termdic)

**Check the results**

In [ ]:
root[0:100]

**Check the numbers of records**

In [ ]:
len(root)

Now, we have the `List` store the `dictionaries`,For checking purpose, packages such as `etree` in `lxml` can be used to find out the numbers of records, compare it to our result, ensure it is correct.

**Create a dictionary to store the list**

In [ ]:
thesaurus = {}
thesaurus["thesaurus"] = root # create dictionary as data structure can be converted to JSON file

In [ ]:
thesaurus

### 2.4. Save the data

From the previous analyzing, we know the `JSON` and `Python` data structure can be converted using a package named `json`, Now we can use the method named `dumps()` to dump our data to `JSON` data

In [ ]:
jsondata = json.dumps(thesaurus)
jsondata

There is a slightly different between our data and the json data, which is the order in the dictionaries, check about the [document](https://docs.python.org/3.6/library/json.html) a variable named `sort_keys` can keep the original order for the data. 

**Try to set the `sort_keys` as `True` and check the results**

In [ ]:
jsondata_ordered = json.dumps(thesaurus, sort_keys=True)
jsondata_ordered

** After check the results, we can store the results to the `json` file, using the `dump()` directly store the `JSON` data to output file.**

In [ ]:
with open('sport.dat', 'w') as outfile:
    json.dump(thesaurus, outfile, sort_keys = True)

## 3. Summary

Based on the above script and process, some conclusions can be drawn:
* XML file sometimes is a well-structured data, but it may contain data not in well-structure. Using thrid party packages can reduce our work to parse the XML file, but we should also know how to use the Regular expression to get the data from the XML file.
- - -
* One of the disadvantage of loading all the file to the string is memory usage, it can cause some problems when using low capacity memory computer.
- - -
* The core concept of this task is to find the data structure for XML,Python,Json file,and convert it.

## 4. References

[python data structures](https://docs.python.org/3.6/tutorial/datastructures.html)
- - -
[tutorial on moodle](https://moodle.vle.monash.edu/course/view.php?id=42906&section=8#8)
- - -
[html package](https://docs.python.org/3.6/library/html.html?highlight=html#module-html)
- - -
[XML](https://en.wikipedia.org/wiki/XML)
- - -
[json](https://en.wikipedia.org/wiki/JSON)
- - -
[json package](https://docs.python.org/2/library/json.html)